# Lecture2
Learning outcomes:
1. Be able to use the chain rule to compute the gradient for composite functions
1. Be able to construct the computational graph for any sequence of computation
1. Understand 


## Backpropagation
 We mentioned in lecture 1 that PyTorch can compute the gradient to any function. To accomplish this, PyTorch uses three concepts:

 
 1. Computational graph
 1. "Database" of derivatives to **primitive** functions
 1. Chain rule
 We explain the three concepts  with an example. Consider the following computations:

 $a=2$
 
 $b=4$
 
 $c=a+b$
 
 $d=\log(a)*\log(b)$

 $e=c*d$

 Where $\log(x)$ is the log base two. 
 
 When a gradient is needed, in this case $\frac{\partial e}{\partial a}$ and $\frac{\partial e}{\partial b}$, PyTorch builds a **computational graph** to perform the operations as shown in the figure below


![Comp-graph-1](comp-graph1.png)

For every **primitive** operation, PyTorch "knows" its derivative. For example, PyTorch has the following rules stored and can look them up when needed.

1. $\frac{\partial\log(x)}{\partial x}=\frac{1}{x*\ln(2)}$
1. $\frac{\partial (x*y)}{\partial x}=y$
1. $\frac{\partial (x*y)}{\partial y}=x$
1. $\frac{\partial (x+y)}{\partial x}=1$
1. $\frac{\partial (x+y)}{\partial y}=1$


Using that information, PyTorch adds, at each step, auxillary nodes to be able to compute the gradient as shown in the figure below.

**NOTE**: Actually the node $d$ involves multiple primitive operations than what is shown (see later) but for now we keep it as it is for simplicity.

![Comp-graph-2](comp-graph2.png)



Suppose that we need to compute the gradient of $e$ with respect to $a$ and $b$. Having the above computational graph, PyTorch uses the chain rule:

 $$\frac{\partial e}{\partial a}=\frac{\partial e}{\partial c}\frac{\partial c}{\partial a}+\frac{\partial e}{\partial d}\frac{\partial d}{\partial a}$$
 $$\frac{\partial e}{\partial b}=\frac{\partial e}{\partial c}\frac{\partial c}{\partial b}+\frac{\partial e}{\partial d}\frac{\partial d}{\partial b}$$

To "Walk backward" in the computational graph to compute the gradients. This means that the value at every node is the product of its parent value with the auxillary node (the contributions of multiple parents are added), as shown in the figure below.

![backprop](backprop.png)

So far, in order to keep the graph simple, we glossed over the fact that $d=\log(a)*\log(b)$ involves **multiple primitive operations**. Below is the breakdown of how PyTorch actually computes the gradient of $d$

![backprop](comp-graph3.png)

The PyTorch code corresponding to the example is shown below. Note 
1. For each **leaf** tensor ```X```, the gradient is stored in ```X.grad```
1. The values for **non-leaf** node ```Y``` is **not** saved, unless ```Y.retain_grad()``` is called

In [ ]:
import torch
a=torch.tensor(2.,requires_grad=True)
b=torch.tensor(4.,requires_grad=True)
c=a+b
d=torch.log(a)*torch.log(b)
e=c*d
e.backward()
print(a.grad,b.grad)